In [1]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer
import statsmodels.nonparametric.kernel_regression as loess

warnings.filterwarnings("ignore")


In [7]:
# Current script folder
current_path = os.getcwd()
dirname = current_path

# location folders
data_in = dirname
data_out = dirname
output = dirname + "/output/"
func = dirname + "/ch00-tech-prep/"
sys.path.append(func)

In [10]:
# Import the prewritten helper functions
from py_helper_functions import *

In [27]:

data_all = pd.read_csv(data_in + "/morg-2014-emp.csv")


In [28]:
data_all

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149311,317051,896679860459501,December,WY,346.2296,692.30,40,39,1,NaN,...,0,0,"Native, Born In US",8,Office supplies and stationery stores (45321),4760,"Private, For Profit",No,No,Employed-At Work
149312,317052,907086820569600,December,WY,294.9800,1984.61,40,44,1,NaN,...,1,3,"Native, Born In US",8,Administration of human resource programs (923),430,Government - State,No,No,Employed-At Work
149313,317053,907086820569600,December,WY,324.1761,2884.61,55,43,1,NaN,...,1,3,"Native, Born In US",8,Nursing care facilities (6231),10,"Private, For Profit",No,No,Employed-At Work
149314,317055,950868097156649,December,WY,321.6982,1153.84,40,42,1,NaN,...,0,0,"Native, Born In US",8,Hospitals (622),5820,"Private, Nonprofit",No,No,Employed-At Work


In [29]:
# SELECT OCCUPATION
# keep only two occupation types: Business Operations Specialists
data_all.loc[
    ((data_all["occ2012"] >= 500) & (data_all["occ2012"] <= 740)), "sample"
] = 1
data_all.loc[data_all["sample"].isna(), "sample"] = 0

In [30]:
data_all = data_all.loc[
    data_all["sample"] == 1, :
].reset_index(drop=True)

In [31]:
data_all["sample"].value_counts()

sample
1.0    3922
Name: count, dtype: int64

In [32]:
data_all["female"] = (data_all.sex == 2).astype(int)
data_all["w"] = data_all["earnwke"] / data_all["uhours"]
data_all["lnw"] = np.log(data_all["w"])
data_all["agesq"] = np.power(data_all["age"], 2)

In [33]:
data_all.to_csv(data_out + "/earnings_inference.csv", index=False)

In [17]:
#####################
# DISTRIBUTION OF EARNINGS
#######################
data_all.loc[:, ["earnwke", "uhours", "w"]].describe()

,earnwke,uhours,w
count,3922.000000,3922.000000,3922.000000
mean,1196.077634,41.022438,28.842154
std,671.841672,8.560748,16.570141
min,0.010000,1.000000,0.000500
25%,720.000000,40.000000,17.933333
50%,1057.000000,40.000000,25.277797
75%,1538.460000,40.000000,36.057500
max,2884.610000,99.000000,461.538000


In [18]:
data_all["female"].value_counts()

female
1    2288
0    1634
Name: count, dtype: int64

In [34]:
data_all.groupby(["occ2012", "female"]).size()

occ2012  female
500      0          26
         1          22
510      0          11
         1           7
520      0          96
         1         114
530      0         128
         1         186
540      0         130
         1         208
565      0         140
         1         151
600      0         115
         1          13
630      0         181
         1         526
640      0          17
         1          59
650      0          61
         1          89
700      0          59
         1          47
710      0         408
         1         336
725      0          28
         1         118
726      0          32
         1          91
735      0         109
         1         172
740      0          93
         1         149
dtype: int64

In [35]:
data_all.groupby(["grade92", "female"]).size()

grade92  female
32       1           2
33       0           1
34       1           1
35       1           2
36       0           1
         1           6
37       0           5
         1           7
38       0           3
         1          10
39       0         187
         1         307
40       0         268
         1         339
41       0          42
         1          81
42       0          81
         1         177
43       0         711
         1         983
44       0         289
         1         326
45       0          22
         1          23
46       0          24
         1          24
dtype: int64

In [53]:
##############################
# Gender gap of logarithm wage is divided into three group based on “grade92”, which means the highest educational grade completed
##############################
data1 = data_all.loc[
    (data_all["grade92"] >= 32) & (data_all["grade92"] <= 36), :
].reset_index(drop=True)
data2 = data_all.loc[
    (data_all["grade92"] >= 37) & (data_all["grade92"] <= 41), :
].reset_index(drop=True)
data3 = data_all.loc[
    (data_all["grade92"] >= 42) & (data_all["grade92"] <= 46), :
].reset_index(drop=True)

In [54]:
reg1 = smf.ols(formula="lnw~female", data=data1).fit(cov_type="HC1")
reg1.summary()

#The result is:
#|t| = 0.068 < 1.96, which means it is not statistically significant.
#p = 0.946 > 0.05, which means it is not statistically significant.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.091
Method:                 Least Squares   F-statistic:                  0.004588
Date:                Sun, 26 Nov 2023   Prob (F-statistic):              0.947
Time:                        12:32:12   Log-Likelihood:                -5.9955
No. Observations:                  13   AIC:                             15.99
Df Residuals:                      11   BIC:                             17.12
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.6210   3.42e-16   7.66e+15      0.000       2.621       2.621
female         0.0093      0.137      0.068      0.946      -0.259       0.277
==============================================================================
Omnibus:                       15.795   Durbin-Watson:                   2.610
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               11.361
Skew:                           1.747   Prob(JB):                      0.00341
Kurtosis:                       5.961   Cond. No.                         4.91
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [43]:
reg2 = smf.ols(formula="lnw~female", data=data2).fit(cov_type="HC1")
reg2.summary()

#The result is:
#|t| = 4.514 > 1.96, which means it is statistically significant.
#p = 0.000 < 0.05, which means it is statistically significant.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     20.38
Date:                Sun, 26 Nov 2023   Prob (F-statistic):           6.96e-06
Time:                        12:29:51   Log-Likelihood:                -911.60
No. Observations:                1249   AIC:                             1827.
Df Residuals:                    1247   BIC:                             1837.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.0725      0.025    124.886      0.000       3.024       3.121
female        -0.1351      0.030     -4.514      0.000      -0.194      -0.076
==============================================================================
Omnibus:                       48.745   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               83.450
Skew:                          -0.308   Prob(JB):                     7.57e-19
Kurtosis:                       4.106   Cond. No.                         2.91
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [44]:
reg3 = smf.ols(formula="lnw~female", data=data3).fit(cov_type="HC1")
reg3.summary()

#The result is:
#|t| = 9.220 > 1.96, which means it is statistically significant.
#p = 0.000 < 0.05, which means it is statistically significant.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     85.02
Date:                Sun, 26 Nov 2023   Prob (F-statistic):           5.85e-20
Time:                        12:30:23   Log-Likelihood:                -2316.5
No. Observations:                2660   AIC:                             4637.
Df Residuals:                    2658   BIC:                             4649.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.4471      0.014    238.457      0.000       3.419       3.475
female        -0.2009      0.022     -9.220      0.000      -0.244      -0.158
==============================================================================
Omnibus:                     2721.180   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           529974.302
Skew:                          -4.529   Prob(JB):                         0.00
Kurtosis:                      71.554   Cond. No.                         2.84
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [57]:
#Overall Gender Gap
reg4 = smf.ols(formula="lnw~female", data=data_all).fit(cov_type="HC1")
reg4.summary()

#The result is:
#|t| = 10.230 > 1.96, which means it is statistically significant.
#p = 0.000 < 0.05, which means it is statistically significant.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     104.7
Date:                Sun, 26 Nov 2023   Prob (F-statistic):           2.92e-24
Time:                        12:39:50   Log-Likelihood:                -3409.0
No. Observations:                3922   AIC:                             6822.
Df Residuals:                    3920   BIC:                             6835.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.3303      0.013    251.136      0.000       3.304       3.356
female        -0.1875      0.018    -10.230      0.000      -0.223      -0.152
==============================================================================
Omnibus:                     3104.647   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           335414.411
Skew:                          -3.082   Prob(JB):                         0.00
Kurtosis:                      47.883   Cond. No.                         2.86
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [58]:
stargazer = Stargazer([reg1, reg2, reg3, reg4])
stargazer.covariate_order(["female","Intercept"])
stargazer.rename_covariates({"Intercept":"Constant"})
stargazer